# Report percent dropped trials

In [4]:
import pandas as pd

In [5]:
import sys
import os
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils 

# Load in subject IDs and paths from CFGLog
config = Config.from_json(configs.CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

# Define preprocessing parameters
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain'])}")
print(sub_ids)

# Initialize dataframe
df = pd.DataFrame(columns=['subject_id', 'num_remaining_epochs', 'num_dropped', 'percent_dropped'])
dropped_epochs = {}

for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        
        # Load in Epochs data
        subject.load_epochs()
        num_epochs = len(subject.epochs)
        subject.load_epochs_info(clbp_processed_path)
        drop_log = subject.drop_log['drop_log'][0] if len(subject.drop_log['drop_log']) > 0 else []
        
        # Save number of remaining epochs, number dropped, and percent dropped to dataframe
        print(f"Subject: {sub_id}, Num Remaining Epochs: {num_epochs - len(drop_log)}, Num Dropped: {len(drop_log)}, Percent Dropped: {len(drop_log)/num_epochs}")
        df.loc[len(df.index)] = [sub_id, num_epochs - len(drop_log), len(drop_log), (len(drop_log)/num_epochs)*100]
        
        # Make a dictionary where the keys are the subject IDs and the values are the dropped epoch indices
        if len(drop_log) > 0:
            dropped_epochs.update({sub_id: drop_log})
            # subject.save(dropped_epochs=dropped_epochs)

Total subjects: 43
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']}
Loaded epochs for subject 018
Subject: 018, Num Remaining Epochs: 73, Num Dropped: 0, Percent Dropped: 0.0
Loaded epochs for subject 022
Subject: 022, Num Remaining Epochs: 52, Num Dropped: 4, Percent Dropped: 0.07142857142857142
Loaded epochs for subject 024
Subject: 024, Num Remaining Epochs: 60, Num Dropped: 0, Percent Dropped: 0.0
Loaded epochs for subject 031
Subject: 031, Num Remaining Epochs: 60, Num Dropped: 0, Percent Dropped: 0.0
Loaded epochs for subject 032
Subject: 032, Num Remaining Epochs: 22, Num Dropped: 19, Percent Dropped: 0.4634146341463415
Loaded epochs for subject 034
Subject: 034, Num Remaining Epochs: 60, Num Dropped: 0,

In [6]:
df.set_index('subject_id', inplace=True)
df

,num_remaining_epochs,num_dropped,percent_dropped
subject_id,,,
018,73,0,0.000000
022,52,4,7.142857
024,60,0,0.000000
031,60,0,0.000000
032,22,19,46.341463
034,60,0,0.000000
036,55,2,3.508772
039,44,8,15.384615
040,52,4,7.142857


In [10]:
# turn dropped epochs dictionary into a saveable xlsx sheet
# make the data type integer
df_dropped_epochs = pd.DataFrame.from_dict(dropped_epochs, orient='index').astype('Int64')


In [11]:
df_dropped_epochs

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
022,14,18,36,38,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
032,0,3,9,22,24,27,28,34,36,38,42,43,44,45,47,48,51,56,59,<NA>
036,52,56,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
039,12,16,31,32,33,50,56,59,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
040,15,24,31,42,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
052,19,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
021,12,29,34,40,41,52,53,72,113,116,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
023,32,33,46,55,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
029,10,30,31,36,37,41,49,53,56,57,58,59,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
037,27,28,30,35,37,38,44,45,46,47,48,49,50,52,54,<NA>,<NA>,<NA>,<NA>,<NA>


In [12]:
# save xlsx sheet
df_dropped_epochs.to_excel('../../data/chronic_low_back_pain/dropped_epochs.xlsx')

In [8]:
# load in ratings master xlsx sheet for all subjects
df_master = pd.read_excel('../../data/chronic_low_back_pain/ratings_master.xlsx')